In [1]:
from pybaseball import *
import numpy as np
import pandas as pd
import pickle
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
# from '../src/cleanids' import *

from numpy import nan
import re

import matplotlib.pyplot as plt

In [153]:
dl_18 = pd.read_pickle('../pickles/filter_dl_18.pkl')
print(len(dl_18))
dl_18

332


,name,team,injury,days,list,startdate,enddate,moneyearned,last,first
0,erichanhold,NYM,Oblique,7,60,2018-09-25,10/1,20510,hanhold,eric
1,michaelfulmer,DET,Knee/Oblique,47,60,2018-09-21,10/110-day,145324,fulmer,michael
2,claybuchholz,TOR,Elbow/Oblique,47,60,2018-09-15,10/110-day,379055,buchholz,clay
3,luizgohara,ATL,Shoulder/Ankle,49,60,2018-09-14,10/110-day,146216,gohara,luiz
4,michaelkopech,CHW,Elbow Tommy John,24,60,2018-09-08,10/1,70320,kopech,michael
5,keurymella,CIN,Oblique,28,60,2018-09-04,10/1,82040,mella,keury
6,josephbiagini,TOR,Oblique,11,10,2018-09-04,9/14,32230,biagini,joseph
8,brockstewart,LAD,Oblique,30,60,2018-09-02,10/1,87900,stewart,brock
9,pablolopez,MIA,Shoulder,31,60,2018-09-01,10/1,90830,lopez,pablo
10,kendallgraveman,CHC,Elbow Tommy John,31,60,2018-09-01,10/1,393328,graveman,kendall


In [57]:
def clean_ids(file):
    ids = pd.read_pickle(file)
    ids['name_last'] = ids['name_last'].str.replace('.', '')
    ids['name_last'] = ids['name_last'].str.replace("'", '')
    ids['name_last'] = ids['name_last'].str.replace(' ', '')
    ids['name_last'] = ids['name_last'].str.strip()
    ids['name_first'] = ids['name_first'].str.replace('.', '')
    ids['name_first'] = ids['name_first'].str.replace("'", '')
    ids['name_first'] = ids['name_first'].str.replace(' ', '')
    ids['name_first'] = ids['name_first'].str.strip()

    ids['name'] = 0
    for idx, val in ids.iterrows():
        ids['name'][idx] = ids['name_first'][idx] + '' + ids['name_last'][idx]

    # ids = ids.drop(['name_last', 'name_first'], axis=1)
    return ids 


In [58]:
keys = clean_ids('../pickles/moreids.pkl')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
def clean_inj(file, year):
    inj = pd.read_csv(file)
    inj['Year'] = year
    inj['Status'] = inj.Status.str[9:]
    DL_df = inj.copy()
    DL_df['List'], DL_df['Status'] = DL_df['Status'].str.split('|', 1).str
    DL_df['Dates'], DL_df['Status'] = DL_df['Status'].str.split('|', 1).str
    DL_df['StartDate'], DL_df['EndDate'] = DL_df['Dates'].str.split('-', 1).str
    DL_df['Dates'] = DL_df['Dates'].str.replace('..-day', '')
    DL_df['MoneyEarned'] = DL_df['Money_Earned'].str[1:]
    DL_df['MoneyEarned'] = DL_df['MoneyEarned'].str.replace(',', '')
    del DL_df['Dates']
    del DL_df['Status']
    del DL_df['Money_Earned']
    DL_df['Player'] = DL_df['Player'].str.lower()
    DL_df['List'] = DL_df['List'].str.replace('-day', '', regex=True)
    dates = []
    df = DL_df
    df.columns = map(str.lower, df.columns)
    for idx, val in df.iterrows():
        val.startdate = str(val.year)+val.startdate
        dates.append(val.startdate)

    dates = [x.rstrip() for x in dates]
    dates = [x.replace(' ','/') for x in dates]
    df['startdate'] = dates
    df['startdate'] = pd.to_datetime(df['startdate'], infer_datetime_format=True)
    
    lasts = []
    firsts = []

    for row in df.iterrows():
        name = row[1][0]
        name = jr_replace(name)
        name = title_replace(name)
        index = row[0]
        first = name.split()[0]
        first = correction2(first).rstrip()
        firsts.append(first)
        last = name.split()[-1]
        lasts.append(last)
    
    df['last'] = lasts
    df['first'] = firsts
    df['player'] = df['player'].str.lower()
    df['player'] = df['player'].str.replace('.', '')
    df['player'] = df['player'].str.replace("'", '')
    df['player'] = df['player'].str.replace(' ', '')
    df.rename(columns={'player': 'name'}, inplace=True)
    
    return df

In [7]:

def jr_replace(x):
    match = re.sub(r'jr$',"",x)
    return match

def title_replace(x):
    match = re.sub(r'iii$',"",x)
    return match

def correction2(string):
    corstr = re.sub('\ +',' ',string)
    final = re.sub('\.','. ',corstr)
    return final

In [154]:
df_18 = clean_inj('../data/dl_list_2018.csv', 2018)

In [155]:
print(len(df_18))
df_18

339


,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first
0,erichanhold,P,NYM,Oblique,7,2018,60,2018-09-25,10/1,20510,hanhold,eric
1,michaelfulmer,SP,DET,Knee/Oblique,47,2018,60,2018-09-21,10/110-day,145324,fulmer,michael
2,claybuchholz,SP,TOR,Elbow/Oblique,47,2018,60,2018-09-15,10/110-day,379055,buchholz,clay
3,luizgohara,SP,ATL,Shoulder/Ankle,49,2018,60,2018-09-14,10/110-day,146216,gohara,luiz
4,michaelkopech,SP,CHW,Elbow Tommy John,24,2018,60,2018-09-08,10/1,70320,kopech,michael
5,keurymella,RP,CIN,Oblique,28,2018,60,2018-09-04,10/1,82040,mella,keury
6,josephbiagini,RP,TOR,Oblique,11,2018,10,2018-09-04,9/14,32230,biagini,joseph
7,josemujica,P,TB,Undisclosed,29,2018,60,2018-09-03,10/1,84970,mujica,jose
8,brockstewart,SP,LAD,Oblique,30,2018,60,2018-09-02,10/1,87900,stewart,brock
9,pablolopez,SP,MIA,Shoulder,31,2018,60,2018-09-01,10/1,90830,lopez,pablo


In [157]:
keys = clean_ids('../pickles/moreids.pkl')
keys

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,name
0,aardsma,david,430911,aardd001,aardsda01,1902,2004.0,2015.0,davidaardsma
1,abad,fernando,472551,abadf001,abadfe01,4994,2010.0,2017.0,fernandoabad
2,achter,aj,592091,achta001,achteaj01,11387,2014.0,2016.0,ajachter
3,adam,jason,592094,adamj002,adamja01,11861,2018.0,2018.0,jasonadam
4,adams,chance,664856,adamc002,adamsch01,18135,2018.0,2018.0,chanceadams
5,adams,austin,613534,adama002,adamsau02,13801,2017.0,2018.0,austinadams
6,adams,austin,542866,adama001,adamsau01,7411,2014.0,2016.0,austinadams
7,adcock,nathan,502264,adcon001,adcocna01,3282,2011.0,2015.0,nathanadcock
8,adleman,tim,534947,adlet001,adlemti01,10968,2016.0,2017.0,timadleman
9,affeldt,jeremy,346793,affej001,affelje01,583,2002.0,2015.0,jeremyaffeldt


In [176]:
merge = df_18.merge(keys, on='name', how='left')
print(len(merge))
to_add = merge[merge['mlb_played_last'].isnull()]
to_add


355


,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last


In [174]:
ip = df_18[df_18['name'].str.contains('zackjones')]
ind = ip.index[0]
ip

,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first
338,zackjones,RP,NYM,Shoulder,30,2018,7,2018-09-01,9/30,83190,jones,zack


In [175]:
df_18 = df_18.drop(index=ind)

In [62]:
def add_to_dict_df():
    add_frame = pd.DataFrame()
    for row in to_add.iterrows():
        name = row[1][0]
        first = row[1][11]
        index = row[0]
        lookup = keys[keys['name'].str.contains(to_add['last'].loc[index])]
        if len(lookup) == 1:
            ins_index = lookup.index[0]
            lookup.set_value(ins_index, 'name_first', first)
            add_frame = add_frame.append(lookup)
    return add_frame

In [93]:
def add_to_dict_manual():
    add_frame = pd.DataFrame()
    for row in to_add.iterrows():
        name = row[1][0]
        first = row[1][11]
        index = row[0]
        lookup = keys[keys['name'].str.contains(to_add['last'].loc[index])]
        print(lookup)
        ins_index = int(input())
        lookup.set_value(ins_index, 'name_first', first)
        add_frame = add_frame.append(lookup)
    return add_frame

In [94]:
add_fr = add_to_dict_manual()

Empty DataFrame
Columns: [name_last, name_first, key_mlbam, key_retro, key_bbref, key_fangraphs, mlb_played_first, mlb_played_last, name]
Index: []
0


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2587: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[index, col] = value


Empty DataFrame
Columns: [name_last, name_first, key_mlbam, key_retro, key_bbref, key_fangraphs, mlb_played_first, mlb_played_last, name]
Index: []
0
    name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
456    germen   gonzalez     542674  germg001  germego01           5876   
480  gonzalez    merandy     642571  gonzm007  gonzame01          17313   
481  gonzalez   severino     605894  gonzs001  gonzase01          14697   
482  gonzalez     miguel     456068  gonzm003  gonzami03           7024   
483  gonzalez     miguel     544838  gonzm004  gonzami04           4049   
484  gonzalez        gio     461829  gonzg003  gonzagi01           7448   
485  gonzalez     miguel     646057  gonzm006  gonzami05          15675   
486  gonzalez     chichi     592346  gonzc002  gonzach01          14663   

     mlb_played_first  mlb_played_last              name  
456            2013.0           2016.0    gonzalezgermen  
480            2018.0           2018.0   merandygonzalez

In [159]:
add_fr

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,name
0,NaN,julian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,joe,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,germen,gonzalez,542674.0,germg001,germego01,5876.0,2013.0,2016.0,gonzalezgermen
480,gonzalez,merandy,642571.0,gonzm007,gonzame01,17313.0,2018.0,2018.0,merandygonzalez
481,gonzalez,severino,605894.0,gonzs001,gonzase01,14697.0,2015.0,2016.0,severinogonzalez
482,gonzalez,miguel,456068.0,gonzm003,gonzami03,7024.0,2012.0,2018.0,miguelgonzalez
483,gonzalez,miguel,544838.0,gonzm004,gonzami04,4049.0,2013.0,2013.0,miguelgonzalez
484,gonzalez,gio,461829.0,gonzg003,gonzagi01,7448.0,2008.0,2018.0,giogonzalez
485,gonzalez,miguel,646057.0,gonzm006,gonzami05,15675.0,2014.0,2014.0,miguelgonzalez
486,gonzalez,chichi,592346.0,gonzc002,gonzach01,14663.0,2015.0,2016.0,chichigonzalez


In [110]:

drop_list = []
for row in to_add.iterrows():
    index = row[0]
    name = row[1][0]
    lookup = df_18[df_18['name'].str.contains(to_add['last'].loc[index])]
    if len(lookup) == 1:
        ins_index = lookup.index[0]
        drop_list.append(ins_index)
#         lookup.set_value(ins_index, 'name_first', first)
#         add_frame = add_frame.append(lookup)
    
#     drop_list.append(index)
drop_list

[282, 296, 315]

In [111]:
df_18 = df_18.drop(index=drop_list)

In [77]:
l = drop_minors()
l

[302, 316, 317, 335, 359]

In [74]:
merge = merge.drop(index=l)

In [64]:
add_frame = add_to_dict_df()
add_frame

""


In [179]:
dups = merge.duplicated(subset='name', keep=False)
print(len(merge[dups]))
merge[dups]

9


,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
87,luisperdomo,SP,SD,Shoulder,38,2018,10,2018-07-28,9/3,115786,perdomo,luis,perdomo,luis,606131,perdl002,perdolu02,14682,2016.0,2018.0
88,luisperdomo,SP,SD,Shoulder,38,2018,10,2018-07-28,9/3,115786,perdomo,luis,perdomo,luis,466412,perdl001,perdolu01,5380,2009.0,2012.0
229,josevaldez,RP,SF,Elbow,133,2018,10,2018-05-22,10/1,389690,valdez,jose,valdez,jose,572308,valdj003,valdejo03,14470,2015.0,2018.0
230,josevaldez,RP,SF,Elbow,133,2018,10,2018-05-22,10/1,389690,valdez,jose,valdez,jose,451990,valdj001,valdejo01,3217,2011.0,2012.0
264,miguelgonzalez,SP,CHW,Undisclosed,162,2018,60,2018-04-23,10/1,4137156,gonzalez,miguel,gonzalez,miguel,456068,gonzm003,gonzami03,7024,2012.0,2018.0
265,miguelgonzalez,SP,CHW,Undisclosed,162,2018,60,2018-04-23,10/1,4137156,gonzalez,miguel,gonzalez,miguel,544838,gonzm004,gonzami04,4049,2013.0,2013.0
266,miguelgonzalez,SP,CHW,Undisclosed,162,2018,60,2018-04-23,10/1,4137156,gonzalez,miguel,gonzalez,miguel,646057,gonzm006,gonzami05,15675,2014.0,2014.0
276,joseramirez,RP,ATL,Shoulder,167,2018,60,2018-04-18,10/1,507346,ramirez,jose,ramirez,jose,542432,ramij004,ramirjo02,10171,2014.0,2018.0
277,joseramirez,RP,ATL,Shoulder,167,2018,60,2018-04-18,10/1,507346,ramirez,jose,ramirez,jose,608070,ramij003,ramirjo01,13510,2013.0,2019.0


In [180]:
merge = merge.drop(index=[277, 266, 265, 230, 88])

In [127]:
new_keys = pd.concat([keys, add_frame])
new_keys = new_keys.sort_values(by='name_last')
new_keys = new_keys.set_index(np.arange(len(new_keys)))
# keys.to_pickle('../pickles/moreids.pkl')

In [178]:
merge = merge[merge['mlb_played_last'] >= 2012 ] 
print(len(merge))

339


In [181]:
merge

,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,erichanhold,P,NYM,Oblique,7,2018,60,2018-09-25,10/1,20510,hanhold,eric,hanhold,eric,621098,hanhe001,hanhoer01,18332,2018.0,2018.0
1,michaelfulmer,SP,DET,Knee/Oblique,47,2018,60,2018-09-21,10/110-day,145324,fulmer,michael,fulmer,michael,605242,fulmm001,fulmemi01,13218,2016.0,2018.0
2,claybuchholz,SP,TOR,Elbow/Oblique,47,2018,60,2018-09-15,10/110-day,379055,buchholz,clay,buchholz,clay,453329,buchc001,buchhcl01,3543,2007.0,2018.0
3,luizgohara,SP,ATL,Shoulder/Ankle,49,2018,60,2018-09-14,10/110-day,146216,gohara,luiz,gohara,luiz,627894,gohal001,goharlu01,15166,2017.0,2018.0
4,michaelkopech,SP,CHW,Elbow Tommy John,24,2018,60,2018-09-08,10/1,70320,kopech,michael,kopech,michael,656629,kopem001,kopecmi01,17282,2018.0,2018.0
5,keurymella,RP,CIN,Oblique,28,2018,60,2018-09-04,10/1,82040,mella,keury,mella,keury,622492,mellk001,mellake01,15964,2017.0,2018.0
6,josephbiagini,RP,TOR,Oblique,11,2018,10,2018-09-04,9/14,32230,biagini,joseph,biagini,joseph,607352,biagj001,biagijo01,13194,2016.0,2019.0
7,josemujica,P,TB,Undisclosed,29,2018,60,2018-09-03,10/1,84970,mujica,jose,mujica,jose,465629,mujie001,mujiced01,3970,2006.0,2017.0
8,brockstewart,SP,LAD,Oblique,30,2018,60,2018-09-02,10/1,87900,stewart,brock,stewart,brock,592779,stewb001,stewabr01,16727,2016.0,2019.0
9,pablolopez,SP,MIA,Shoulder,31,2018,60,2018-09-01,10/1,90830,lopez,pablo,lopez,pablo,641154,lopep002,lopezpa01,17085,2018.0,2019.0


In [182]:
print(len(merge))

334


In [183]:
merge.to_pickle('../pickles/2018_inj_agg_games.pkl')

In [185]:
merge[merge['moneyearned'].isnull()]

,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
